# ワークフロー実行のための準備

このノートブックでは、ワークフロー実行のための準備を行います。 
上から順番にセルを実行してください。

## 1. .gitconfigにユーザー名とメールアドレスを登録する

    1. 以下のセルでgitアカウントのユーザー名とメールアドレスを入力してください。

In [ ]:
name = input()

In [ ]:
email = input()

    2. 入力されたユーザー名とメールアドレスを.gitconfigに登録します。

In [ ]:
import os

os.system("cd /home/jovyan")
os.system("git config --global user.name " + name)
os.system("git config --global user.email " + email)

## 2. Dataladを使用するため、このリポジトリをデータセットとする

In [ ]:
%%sh
cd ~/
datalad create --force

## 3. ワークフロー実行結果をGINリポジトリに反映させるため、GINリポジトリをリモートリポジトリに設定する

In [ ]:
%%sh
mkdir ./.tmp
git remote show origin > ./.tmp/remote.txt

In [ ]:
import json
json_open = open('/home/jovyan/WORKFLOW/param_files/param_for_dataladsiblings.json', 'r')
json_load = json.load(json_open)
http = json_load['http']
ssh = json_load['ssh']

In [ ]:
with open('./.tmp/remote.txt', 'r') as f:
    lines = [s.strip() for s in f.readlines()]
    ssh_push_url = lines[2].replace(http, ssh) 

In [ ]:
from datalad import api

try:
    api.siblings(action='add', name='gin', url=ssh_push_url)
except:
    pass

## 4. Git管理対象外ファイルを.gitignoreで設定する

In [ ]:
%%sh

echo "
/.cache/
/.conda/
/.config/
/.ipython/
/.local/
/.tmp/
/.WORKFLOW/
/.bashrc
/.bash_logout
/.profile
/.netrc
.ipynb_checkpoints/
" > ~/.gitignore

## 5. 実験用フォルダを作成し、その中にpipeline.jsonを用意する

In [ ]:
%%sh
mkdir ~/experiments
echo "[]" > ~/experiments/pipeline.json

## 6. 必要なファイルをPACKAGEテンプレートに複製する

In [ ]:
%%sh
cd ~/
cp .gitconfig WORKFLOW/PACKAGE/.gitconfig
cp environment.yml WORKFLOW/PACKAGE/environment.yml

## 7. 変更内容を書き戻す

In [ ]:
%%sh

echo "
machine dg02.dg.rcos.nii.ac.jp
login 【ユーザ名】
password 【パスワード】
" > ~/.netrc

cd ~/
git add .
git commit -m 'save'
git push origin master